# 네이버 영화평(알라딘)에 나온 형태소 SQLite DB에 저장
- 영화평에 나온 형태소 추출
- Okt 형태소 분석
- 형태소, 품사, 나온횟수 DB에 저장하기

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [3]:
df = pd.read_excel('../01_TextMining/data/04.NaverMovie_Aladdin.xlsx')
df.drop(df.columns[0], axis='columns', inplace=True)
df.head()

,point,review,date
0,10,윌 스미스가 하드캐리하여 의도치 않게 원작을 뛰어넘은 실사영화,2019.05.23 11:26
1,10,영화가 너무 아름답다..ost도 잊혀지지 안고 디즈니 실사중에서 최고중에 최고다 쟈...,2019.05.23 09:11
2,10,존잼임... 난 처음 도입부에 아~라~비안 나~잇 이 부분에서부터 감동받아서 울 뻔...,2019.05.23 21:57
3,10,자스민의 speechless 명장면이고 덤으로 윌 스미스의 연기력 또한 매우 굿~~,2019.05.23 12:55
4,10,A whole new world 나올때때 개소름...,2019.05.23 09:06


In [4]:
reviews = df['review']
reviews.shape

(26440,)

In [5]:
conn = sqlite3.connect('./test.db') 
cur = conn.cursor()

In [6]:
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Movie_reviews \
    (word TEXT, \
     pos TEXT, \
     count INT, \
     PRIMARY KEY(word, pos));')

In [7]:
from konlpy.tag import Okt
okt = Okt()

C:\ProgramData\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [8]:
morphs = okt.pos(str(reviews[0]))
morphs

[('윌', 'Noun'),
 ('스미스', 'Noun'),
 ('가', 'Josa'),
 ('하드캐리', 'Noun'),
 ('하여', 'Verb'),
 ('의도', 'Noun'),
 ('치', 'Noun'),
 ('않게', 'Verb'),
 ('원작', 'Noun'),
 ('을', 'Josa'),
 ('뛰어넘은', 'Verb'),
 ('실사영화', 'Noun')]

In [10]:
# Test용 데이터 입력
cur.execute("INSERT INTO Movie_reviews VALUES('윌', 'Noun', 1);")
conn.commit()

In [30]:
# Test용 데이터 조회
sql = "SELECT count FROM Movie_reviews WHERE word like ? and pos like ?;"
cur.execute(sql, ('윌', 'Noun'))
count, = cur.fetchone()
print(count)

704


In [27]:
# Test용 데이터 삭제
cur.execute("DELETE FROM Movie_reviews WHERE word like '윌';")
conn.commit()

In [15]:
def countMorph(word, pos):    
    count = 0
    try:
        cur = conn.cursor()
        sql = "SELECT count FROM Movie_reviews WHERE word like ? and pos like ?"
        cur.execute(sql, (word, pos))
        count, = cur.fetchone()
    except TypeError as te:
        count = 0
    except Exception as e: 
        print('예외가 발생했습니다.', e)
        count = -1
    finally:
        return count

In [21]:
print(countMorph('윌', 'Noun'))
print(countMorph('원작', 'Noun'))
x = ('윌', 'Noun')
countMorph(*x)

0
0


0

In [22]:
def insertMorph(word, pos, count):
    cur = conn.cursor()
    sql = "INSERT INTO Movie_reviews VALUES(?, ?, ?);"
    cur.execute(sql, (word, pos, count))
    conn.commit()

In [23]:
insertMorph(*x, 1)

In [25]:
def updateMorph(word, pos, count):
    cur = conn.cursor()
    sql = "UPDATE Movie_reviews SET count=? WHERE word like ? and pos like ?;"
    cur.execute(sql, (count, word, pos))
    conn.commit()

In [26]:
updateMorph(*x, 2)

In [29]:
i = 1
for review in reviews:
    if i % 1000 == 0:
        print(i, end=' ')
    i += 1
    morphs = okt.pos(str(review))
    for morph in morphs:
        if morph[1] in ["Josa", "Eomi", "Punctuation"]:
            continue
        count = countMorph(*morph)     # Unpacking
        if count == 0:                 # 신규 등록
            insertMorph(*morph, 1)
        elif count > 0:                # count update
            updateMorph(*morph, count+1)
        else:
            print('DB Error!!!')

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 

In [31]:
cur = conn.cursor()
sql = "SELECT count(*) FROM Movie_reviews;"
cur.execute(sql)
count, = cur.fetchone()
print(count)

23977


In [32]:
cur = conn.cursor()
sql = "SELECT * FROM Movie_reviews ORDER BY count DESC LIMIT 10;"
cur.execute(sql)
rows = cur.fetchall()
for row in rows:
    print(row)

('영화', 'Noun', 9609)
('너무', 'Adverb', 7616)
('노래', 'Noun', 3302)
('진짜', 'Noun', 3276)
('최고', 'Noun', 3273)
('들', 'Suffix', 3063)
('알라딘', 'Noun', 2897)
('지니', 'Noun', 2404)
('윌스미스', 'Noun', 2320)
('디즈니', 'Noun', 2246)


In [33]:
sql = "SELECT * FROM Movie_reviews;"
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();
columnName = ['형태소', '품사', '등장횟수']
df = pd.DataFrame(columns = columnName)
for row in rows:
    df = df.append(pd.DataFrame([list(row)], columns = columnName), 
                   ignore_index=True)
df.head()

,형태소,품사,등장횟수
0,윌,Noun,704
1,스미스,Noun,810
2,하드캐리,Noun,405
3,하여,Verb,33
4,의도,Noun,18


In [48]:
conn.close()

In [35]:
setMorphs = set(df['형태소'])
len(setMorphs)

23677

In [42]:
# 같은 형태소, 다른 품사
length = len(df)
zeros = np.zeros(length, dtype = np.int32)
df['동일형태소'] = zeros.T

In [39]:
df.head()

,형태소,품사,등장횟수,동일형태소
0,윌,Noun,704,0
1,스미스,Noun,810,0
2,하드캐리,Noun,405,0
3,하여,Verb,33,0
4,의도,Noun,18,0


In [44]:
for i in range(length-1):
    if i % 100 == 0:
        print(i, end=' ')
    if df.iloc[i,3] == 1:
        continue;
    for k in range(i+1, length):
        if df.iloc[i,0] == df.iloc[k,0]:
            df.iloc[i,3] = 1
            df.iloc[k,3] = 1
            continue

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 17800 17900 18000 18100 18200 18300 18400 18

In [45]:
ddf = df[df['동일형태소'] == 1]

In [46]:
ddf

,형태소,품사,등장횟수,동일형태소
10,영화,Noun,9609,1
14,도,Noun,648,1
19,중,Suffix,670,1
28,울,Modifier,62,1
36,아,Exclamation,288,1
...,...,...,...,...
22671,이었다면,Foreign,1,1
22871,넘은게,Foreign,1,1
22882,그만,Modifier,2,1
23191,짓,VerbPrefix,2,1


In [47]:
ddf[ddf['형태소'] == '영화']

,형태소,품사,등장횟수,동일형태소
10,영화,Noun,9609,1
15265,영화,Foreign,1,1


In [50]:
df.to_csv('data/03.영화평 형태소.csv', encoding='euc-kr')